##Import Libraries

In [ ]:
!pip install spacy
!pip install pysrt #per sottotitoli

In [ ]:
#find top words
def top_frequent(text, num_words):
  #frequency of most common words
  import spacy
  from collections import Counter

  nlp = spacy.load("en")
  text = text

  #lemmatization
  doc = nlp(text)
  token_list = list()
  for token in doc:
    #print(token, token.lemma_)
    token_list.append(token.lemma_)
  token_list

  lemmatized = ''
  for _ in token_list:
    lemmatized = lemmatized + ' ' + _
  lemmatized

  #remove stopwords and punctuations
  doc = nlp(lemmatized)
  words = [token.text for token in doc if token.is_stop != True and token.is_punct != True]
  word_freq = Counter(words)
  common_words = word_freq.most_common(num_words)
  return common_words

##Total Course

In [ ]:
#   parse sub
#total words of JP in the course
import pysrt

total_string = ''
for text_n in range(1, 10):
  subs = pysrt.open('/content/drive/My Drive/Colab Notebooks/Projects/20200729_NLP_frequency/JP_'+str(text_n)+'.srt')
  for sub in subs:
    #print(sub.text)
    total_string += sub.text
total_string

In [ ]:
len(total_string)

In [ ]:
common_words = top_frequent(total_string, 1000)
common_words

##Individual Lessons

In [ ]:
#https://www.twinword.com/api/language-scoring.php
#Language Scoring

In [ ]:
def find_common_factors(X):
  #flat everything in one list
  one_row = list()
  for k in X.values:
    for n in k:
      one_row.append(n)
  one_row = pd.DataFrame(one_row)
  one_row

  #labeled list
  from sklearn.preprocessing import LabelEncoder
  le = LabelEncoder()
  le.fit(one_row[0])
  one_row_labeled = le.transform(one_row[0])
  one_row_labeled

  #rebuild the original dataset and convert it to DataFrame
  import numpy
  X_labeled = numpy.array_split(one_row_labeled, (X.shape[0]))
  X_labeled = pd.DataFrame(X_labeled)
  X_labeled

  m = [[0 for x in range(max(one_row_labeled)+1)] for x in range(len(X_labeled))]

  #turn each corresponding label to 1
  for row in range(len(X_labeled.values)):
    for num in range(len(X_labeled.values[1])):
      m[row][X_labeled.values[row][num]] = 1
  m = pd.DataFrame(m)
  m

  original_shape = m.shape
  original_shape

  #convert column num in corresponding text
  m.loc[original_shape[0]] = [0 for x in range(0, original_shape[1])]
  m

  for _ in range(original_shape[0]):
    m.loc[original_shape[0]] += m.loc[_] 
  m

  #rename columns with label values
  m.columns = a = le.inverse_transform(m.columns)
  m

  #isolate count of columns
  m.loc[m.shape[0]] = m.columns #we add last col with names
  f = m.transpose().drop([x for x in range(m.shape[0]-2)], axis=1).transpose() #we only save the last 2 cols
  f = f.reset_index()
  f.pop('index')

  #sorting values
  f = f.transpose().reset_index().drop(['index'], axis=1)
  return f.sort_values(0, axis=0, ascending=False, inplace=False, kind='quicksort', na_position='last')

In [ ]:
#parse individual lesson
#total words of JP in the course
import pysrt
import pandas as pd

total_subs = list()
for text_n in range(1, 15):
  subs = pysrt.open('/content/drive/My Drive/Colab Notebooks/Projects/20200729_NLP_frequency/JP_'+str(text_n)+'.srt')
  top_words = top_frequent(subs.text, 12) #n_words qui, metto 12 perchè le prime 2 si eliminano
  df = pd.DataFrame([top_words[x][0] for x in range(10)])
  total_subs.append(df)

X = pd.concat(total_subs, axis=1)
X = X.transpose()
X

In [ ]:
find_common_factors(X).reset_index().drop(['index'], axis=1)